# Experiments in Extreme Value Theory (EVT)

## Dependencies

In [4]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

from IPython.core.pylabtools import figsize

import numpy as np
import pandas as pd

import scipy as sp
#from scipy.optimize import minimize
from scipy.stats import genextreme

print(f"    Pandas {pd.__version__}")
print(f"     NumPy {np.__version__}")
print(f"MatPlotLib {matplotlib.__version__}")
print(f"     SciPy {sp.__version__}")

    Pandas 1.0.3
     NumPy 1.18.1
MatPlotLib 3.1.3
     SciPy 1.4.1


## References

[Cole 2001] Stuart Cole, <i>"An Introduction to Statistical Modeling of Extreme Values"</i>, 2001.

## The Generalized Extreme Value (GEV) Distribution

The GEV family of probability densities is described as follows in [Cole 2001]:

$G(z;\mu,\sigma,\xi) = \exp \left( - \left[ 1 + \xi \left( \frac {z - \mu} {\sigma} \right) \right]^{-1/\xi} \right)$

<p>where
<p>$-\infty < \mu < \infty$,
<p>$\sigma > 0$,
<p>$-\infty < \xi < \infty$,
<p>and $z$ is such that $1 + \xi(z - \mu)/\sigma > 0$

NOTE: The shape parameter, $\xi$, in <b><i>scipy.stats.genextreme</i></b> is the <u><b>negative</b></u> of the shape parameter in R's <i><b>ismev</b></i> library.

SciPy.stats.genextreme, a generalized extreme value continuous random variable, is [documented here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.genextreme.html).

## Example 1

In [76]:
from scipy.stats import norm

mean = 100.0
sdev = 10.0

rv = norm(loc=mean, scale=sdev)  # "Frozen" random variable

In [86]:
n = 100  # Number of samples
m = 12  # Size of each sample

samples = [rv.rvs(size=m) for _ in range(n)]
samples[:3] # First 3 samples

[array([ 92.39634534,  94.63432949,  97.31140871, 111.34006002,
         99.90648059, 102.93486475, 106.02887502, 107.65922535,
         92.87090988, 107.69981059, 106.66010105,  89.13489403]),
 array([112.66953198, 103.17911184, 105.98214517, 101.27752148,
        110.93833794,  88.71093326,  97.99905021,  97.67042753,
        110.4119903 , 105.81347582,  93.49825699,  80.6428133 ]),
 array([104.32832255,  95.92392991,  92.15734349, 110.29228442,
         70.28643683, 106.57577815, 104.80026381,  96.46463945,
         91.61594917, 113.4476411 , 107.27508692,  99.67071816])]

In [87]:
maxes = [np.max(sample) for sample in samples]
maxes[:3]  # First 3 maximums of samples

[111.3400600199686, 112.6695319831456, 113.44764109587392]